# MCP in OpenAI Agents SDK
1. Create a Client
2. Have it spawn a server
3. Collect the tools that server can use

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

os.environ["PATH"] = "/home/sngo/.nvm/versions/node/v22.20.0/bin:/home/sngo/.local/bin:" + os.environ["PATH"]


In [ ]:
import shutil
shutil.which("uv")
print(os.environ["PATH"])

In [ ]:
load_dotenv(override=True)

In [ ]:
# run uvx command
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

## 2 more MCP Servers - Javascript based with node

In [ ]:
# do this to make the cell can run npx command
#os.environ["PATH"] = "/home/sngo/.nvm/versions/node/v22.20.0/bin:" + os.environ["PATH"]

In [ ]:
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

### Bring on the Agent with Tools!

In [ ]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

# do both in same block to ensure both contexts are active at the same time
async with ( 
    MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files,
    MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser,
):
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)

As always check trace: https://platform.openai.com/traces